In [1]:
%cd ../../ 

/mnt/working/futurama/pytracking


In [2]:
import os
import sys
import numpy as np
import pandas as pd
from collections import OrderedDict
from pytracking.evaluation import Tracker, get_dataset, trackerlist
from pytracking.analysis.extract_results import calc_seq_err_robust, calc_err_center, calc_iou_overlap
import torch
import json

/mnt/working/miniconda/envs/pytracking/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def load_results(result_file):
    """
    加载跟踪结果文件
    """
    try:
        with open(result_file, 'r') as f:
            lines = f.readlines()
        
        results = []
        for line in lines:
            if line.strip():
                # 解析边界框 [x, y, w, h]
                parts = line.strip().split()
                if len(parts) >= 4:
                    bbox = [float(x) for x in parts[:4]]
                    results.append(bbox)
        
        return torch.tensor(results).float()
    except Exception as e:
        print(f"读取结果文件 {result_file} 时出错: {e}")
        return torch.zeros(0, 4)

def calculate_metrics(pred_bb, anno_bb):
    """
    计算成功率和精确率指标
    """
    if len(pred_bb) == 0 or len(anno_bb) == 0:
        return 0.0, 0.0
    
    # 确保长度一致
    min_len = min(len(pred_bb), len(anno_bb))
    pred_bb = pred_bb[:min_len]
    anno_bb = anno_bb[:min_len]
    
    # 计算IoU重叠
    iou_overlap = calc_iou_overlap(pred_bb, anno_bb)
    
    # 计算中心点误差
    center_error = calc_err_center(pred_bb, anno_bb)
    
    # 成功率：IoU > 0.5的帧数比例
    success_rate = (iou_overlap > 0.5).float().mean().item()
    
    # 精确率：中心点误差 < 20像素的帧数比例  
    precision_rate = (center_error < 20).float().mean().item()
    
    return success_rate, precision_rate

def get_sequence_attributes(sequence_name:str):   
    sequence_name = sequence_name[sequence_name.find("_")+1:]
    try:
        with open("/home/zhangshuo2024/futurama/pytracking/pytracking/evaluation/dataset_attribute_specs/UAV20L_attributes.json", "r") as f:
            attributes = json.load(f)
            return attributes[sequence_name]
    except:
        print("读取数据集的属性出错", sequence_name)
        return []

In [4]:
def generate_latex_table(df):
    """
    生成LaTeX格式的表格
    """
    latex_file = 'uav20l_results.tex'
    
    with open(latex_file, 'w') as f:
        f.write("\\begin{table}[h]\n")
        f.write("\\centering\n")
        f.write("\\caption{UAV20L基准测试结果}\n")
        f.write("\\begin{tabular}{" + "c" * len(df.columns) + "}\n")
        f.write("\\hline\n")
        
        # 表头
        f.write(" & ".join(df.columns) + " \\\\\n")
        f.write("\\hline\n")
        
        # 数据行
        for _, row in df.iterrows():
            f.write(" & ".join(str(x) for x in row) + " \\\\\n")
        
        f.write("\\hline\n")
        f.write("\\end{tabular}\n")
        f.write("\\end{table}\n")
    
    print(f"LaTeX表格已保存到: {latex_file}")


In [5]:
def generate_performance_table(tracker_results, attributes):
    """
    生成性能表格
    """
    print("\n" + "="*100)
    print("UAV20L 基准测试结果")
    print("="*100)
    
    # 准备表格数据
    table_data = []
    
    # 表头
    header = ['Trackers', 'Overall Success', 'Overall Precision']
    for attr_name in attributes.keys():
        header.extend([f'{attributes[attr_name]} Suc.', f'{attributes[attr_name]} Pre.'])
    
    # 每个跟踪器的数据
    for tracker_name, results in tracker_results.items():
        row = [tracker_name,
               f"{results['overall_success']*100:.1f}",
               f"{results['overall_precision']*100:.1f}"]
        
        for attr_name, attr_short in attributes.items():
            attr_result = results['attributes'][attr_short]
            row.extend([f"{attr_result['success']*100:.1f}",
                       f"{attr_result['precision']*100:.1f}"])
        
        table_data.append(row)
    
    # 创建DataFrame并打印
    df = pd.DataFrame(table_data, columns=header)
    print(df.to_string(index=False))
    
    # 保存到文件
    output_file = 'uav20l_evaluation_results.csv'
    df.to_csv(output_file, index=False)
    print(f"\n结果已保存到: {output_file}")
    
    # 生成LaTeX表格格式
    generate_latex_table(df)

In [6]:
def analyze_results(trackers, dataset):
    """
    分析跟踪结果并生成性能表格
    """
    # 定义属性类别
    attributes = {
        'Illumination Variations': 'IV',
        'Viewpoint Change': 'VC', 
        'Similar Objects': 'SOB',
        'Scale Variation': 'SV',
        'Fast Motion': 'FM',
        'Full Occlusion': 'FOC',
        'Low Resolution': 'LR'
    }
    
    # 存储每个跟踪器的结果
    tracker_results = {}
    
    for tracker in trackers:
        print(f"分析跟踪器: {tracker.name}")
        
        # 初始化结果字典
        tracker_results[tracker.name] = {
            'overall_success': 0,
            'overall_precision': 0,
            'attributes': {}
        }
        
        # 为每个属性初始化
        for attr_name, attr_short in attributes.items():
            tracker_results[tracker.name]['attributes'][attr_short] = {
                'success': 0,
                'precision': 0
            }
        
        # 计算每个序列的结果
        sequence_results = []
        
        for seq in dataset:
            try:
                # 读取跟踪结果
                result_file = os.path.join(tracker.results_dir, f'{seq.name}.txt')
                if not os.path.exists(result_file):
                    continue
                
                # 加载预测边界框
                pred_bb = load_results(result_file)
                
                # 获取真实边界框
                anno_bb = torch.tensor(seq.ground_truth_rect).float()
                
                # 计算成功率和精确率
                success_rate, precision_rate = calculate_metrics(pred_bb, anno_bb)
                
                sequence_results.append({
                    'sequence': seq.name,
                    'success': success_rate,
                    'precision': precision_rate,
                    'attributes': get_sequence_attributes(seq.name)
                })
                
            except Exception as e:
                print(f"处理序列 {seq.name} 时出错: {e}")
                continue
        
        # 计算整体性能
        if sequence_results:
            overall_success = np.mean([r['success'] for r in sequence_results])
            overall_precision = np.mean([r['precision'] for r in sequence_results])
            
            tracker_results[tracker.name]['overall_success'] = overall_success
            tracker_results[tracker.name]['overall_precision'] = overall_precision
            
            # 计算每个属性的性能
            for attr_name, attr_short in attributes.items():
                attr_results = [r for r in sequence_results if attr_short in r['attributes']]
                if attr_results:
                    attr_success = np.mean([r['success'] for r in attr_results])
                    attr_precision = np.mean([r['precision'] for r in attr_results])
                    
                    tracker_results[tracker.name]['attributes'][attr_short] = {
                        'success': attr_success,
                        'precision': attr_precision
                    }
    
    # 生成表格
    generate_performance_table(tracker_results, attributes)

In [9]:
def run_evaluation():
    """
    运行UAV20L基准测试评估
    """
    print("开始UAV20L基准测试评估...")
    
    # 1. 定义要评估的跟踪器列表
    # 这里需要根据您的实际跟踪器进行修改
    trackers = []
    
    # 添加TaMOs跟踪器（使用您训练的模型）
    trackers.extend(trackerlist('tamos', 'tamos_swim_base', range(1,2)))
    
    # 添加其他基准跟踪器进行对比
    # 注意：这些跟踪器需要预先训练或下载
    # try:
    #     trackers.extend(trackerlist('atom', 'default', range(1)))
    #     trackers.extend(trackerlist('dimp', 'dimp18', range(1)))
    #     trackers.extend(trackerlist('eco', 'default', range(1)))
    # except:
    #     print("警告：部分基准跟踪器未找到，将只评估可用的跟踪器")
    
    # 2. 获取UAV数据集
    dataset = get_dataset('uav20l')
    
    # 3. 运行评估
    print(f"将在{len(dataset)}个序列上评估{len(trackers)}个跟踪器...")
    
    # 存储结果
    results = {}
    
    for tracker in trackers:
        print(f"评估跟踪器: {tracker.name}")
        
        tracker.results_dir = "/home/zhangshuo2024/futurama/pytracking/pytracking/tracking_results/tamos/tamos_swin_base_001"
    
    # 4. 分析结果
    print("开始分析结果...")
    analyze_results(trackers, dataset)

In [10]:
print("UAV20L基准测试评估工具")
print("="*50)
    
# 检查环境
try:
    from pytracking.evaluation import get_dataset
    dataset = get_dataset('uav20l')
    print(f"数据集加载成功，包含{len(dataset)}个序列")
except Exception as e:
    print(f"数据集加载失败: {e}")
    print("请确保已正确配置UAV数据集路径")
    sys.exit(1)

# 运行评估
try:
    run_evaluation()
except KeyboardInterrupt:
    print("\n评估被用户中断")
except Exception as e:
    print(f"评估过程中出现错误: {e}")
    import traceback
    traceback.print_exc()

UAV20L基准测试评估工具
数据集加载成功，包含20个序列
开始UAV20L基准测试评估...
将在20个序列上评估1个跟踪器...
评估跟踪器: tamos
开始分析结果...
分析跟踪器: tamos

UAV20L 基准测试结果
Trackers Overall Success Overall Precision IV Suc. IV Pre. VC Suc. VC Pre. SOB Suc. SOB Pre. SV Suc. SV Pre. FM Suc. FM Pre. FOC Suc. FOC Pre. LR Suc. LR Pre.
   tamos            47.1              58.5    35.4    48.3    57.4    64.4     42.3     49.8    45.1    56.4    48.1    57.8     28.3     48.0    24.5    44.1

结果已保存到: uav20l_evaluation_results.csv
LaTeX表格已保存到: uav20l_results.tex
